In [1]:

# Standard setup: imports and helper functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import os

RANDOM_STATE = 42

def evaluate_and_store(clf, X_train, y_train, X_test, y_test, results, label):
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    results[label] = {
        'model': clf,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'y_test_pred': y_test_pred,
        'y_train_pred': y_train_pred
    }
    return results


In [2]:

# Dataset 1: Heart Disease (Cleveland)
# Download link (raw processed):
# https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

heart_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
# Column names based on UCI description (subset commonly used)
cols = ["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","target"]
try:
    heart = pd.read_csv(heart_url, header=None, names=cols, na_values='?')
except Exception as e:
    print('Could not download heart dataset automatically in this environment. Please run this notebook in an environment with internet to download the UCI files.')
    heart = pd.DataFrame()  # placeholder

# Quick preprocessing (if data present)
if not heart.empty:
    heart = heart.dropna()  # drop rows with missing data for simplicity
    # Convert target to binary: 0 (no disease) vs 1 (disease present)
    heart['target'] = heart['target'].apply(lambda x: 0 if x==0 else 1)
    X1 = heart.drop('target', axis=1)
    y1 = heart['target']
    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=RANDOM_STATE, stratify=y1)
else:
    X1_train = X1_test = y1_train = y1_test = None


In [3]:

# Dataset 2: Breast Cancer Wisconsin (Diagnostic)
# Raw file: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
wdbc_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
try:
    wdbc = pd.read_csv(wdbc_url, header=None)
except Exception as e:
    print('Could not download wdbc dataset automatically in this environment. Please run this notebook in an environment with internet to download the UCI files.')
    wdbc = pd.DataFrame()

if not wdbc.empty:
    # According to UCI: column 1 = ID, 2 = diagnosis (M/B), 3-32 = 30 real-valued features
    X2 = wdbc.iloc[:, 2:]
    y2 = wdbc.iloc[:, 1].map({'M':1, 'B':0})
    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=RANDOM_STATE, stratify=y2)
else:
    X2_train = X2_test = y2_train = y2_test = None


In [4]:

# Training Decision Trees for both datasets with the four configurations:
# 1) Gini without pruning
# 2) Gini with pruning (ccp_alpha=0.015)
# 3) Entropy without pruning
# 4) Entropy with pruning (ccp_alpha=0.015)

results = {}

def run_all_for_dataset(X_train, y_train, X_test, y_test, prefix):
    res = {}
    # Gini without pruning
    clf_gini = DecisionTreeClassifier(criterion='gini', random_state=RANDOM_STATE)
    res = evaluate_and_store(clf_gini, X_train, y_train, X_test, y_test, res, prefix + '_gini_no_prune')
    # Gini with pruning
    clf_gini_p = DecisionTreeClassifier(criterion='gini', random_state=RANDOM_STATE, ccp_alpha=0.015)
    res = evaluate_and_store(clf_gini_p, X_train, y_train, X_test, y_test, res, prefix + '_gini_prune')
    # Entropy without pruning
    clf_ent = DecisionTreeClassifier(criterion='entropy', random_state=RANDOM_STATE)
    res = evaluate_and_store(clf_ent, X_train, y_train, X_test, y_test, res, prefix + '_entropy_no_prune')
    # Entropy with pruning
    clf_ent_p = DecisionTreeClassifier(criterion='entropy', random_state=RANDOM_STATE, ccp_alpha=0.015)
    res = evaluate_and_store(clf_ent_p, X_train, y_train, X_test, y_test, res, prefix + '_entropy_prune')
    return res

if X1_train is not None:
    results.update(run_all_for_dataset(X1_train, y1_train, X1_test, y1_test, 'dataset1'))

if X2_train is not None:
    results.update(run_all_for_dataset(X2_train, y2_train, X2_test, y2_test, 'dataset2'))

# Print accuracy table
import pprint
pp = pprint.PrettyPrinter(indent=2)
acc_table = []
for k,v in results.items():
    acc_table.append({
        'model': k,
        'train_acc': v['train_acc'],
        'test_acc': v['test_acc']
    })
acc_df = pd.DataFrame(acc_table)
print('\nAccuracy Table:\n')
display(acc_df)



Accuracy Table:



,model,train_acc,test_acc
0,dataset1_gini_no_prune,1.000000,0.733333
1,dataset1_gini_prune,0.913043,0.733333
2,dataset1_entropy_no_prune,1.000000,0.744444
3,dataset1_entropy_prune,0.966184,0.766667
4,dataset2_gini_no_prune,1.000000,0.900585
5,dataset2_gini_prune,0.949749,0.906433
6,dataset2_entropy_no_prune,1.000000,0.941520
7,dataset2_entropy_prune,0.989950,0.935673


In [7]:
csv_path = "/content/cancer patient data sets.csv"
print("Loading dataset from:", csv_path)
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    print("Error loading dataset:", e)
    df = pd.DataFrame()

if df.empty:
    print("Dataset empty or couldn't be loaded. Please ensure the file exists at the given path.")
else:
    print("\nFirst 5 rows:")
    display(df.head())
    print("\nDataset shape:", df.shape)
    print("\nColumns and dtypes:")
    display(df.dtypes)

    # Check target 'Label' existence
    target_col = None
    for col in df.columns:
        if col.lower()=='label':
            target_col = col
            break
    if target_col is None:
        # Try common names
        if 'Label' in df.columns:
            target_col = 'Label'
        elif 'label' in df.columns:
            target_col = 'label'
    print("Detected target column:", target_col)

    # Basic EDA
    print("\nMissing values per column:")
    display(df.isnull().sum())
    print("\nAny duplicate rows? ->", df.duplicated().sum())
    if df.duplicated().sum()>0:
        print("Dropping duplicates...")
        df = df.drop_duplicates().reset_index(drop=True)

    # Check label balance
    if target_col:
        print("\nLabel distribution:")
        display(df[target_col].value_counts())
        print("\nLabel distribution (percentage):")
        display(df[target_col].value_counts(normalize=True).round(3))
    else:
        print("No explicit 'Label' target found; attempting to infer a binary target column named 'target' or similar.")
        # try to infer
        for guess in ['target','Target','Outcome','diagnosis','Diagnosis']:
            if guess in df.columns:
                target_col = guess
                print("Inferred target column:", target_col)
                break
    # Identify categorical features
    cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
    # exclude target if it's object but is the label
    if target_col in cat_cols:
        cat_cols_no_target = [c for c in cat_cols if c!=target_col]
    else:
        cat_cols_no_target = cat_cols
    print("\nCategorical columns (excluding target):", cat_cols_no_target)

    # If categorical features present, show unique values
    for c in cat_cols_no_target:
        print(f"Unique values for {c} ({df[c].nunique()}): {df[c].unique()[:10]}")

    # Simple handling: if categorical and low-cardinality convert to numeric via label encoding
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_proc = df.copy()
    for c in cat_cols_no_target:
        df_proc[c] = le.fit_transform(df_proc[c].astype(str))
        print(f"Encoded {c} with LabelEncoder.")

    # If target is categorical strings, encode to 0/1
    if target_col and df_proc[target_col].dtype=='object':
        df_proc[target_col] = le.fit_transform(df_proc[target_col].astype(str))
        print(f"Encoded target column {target_col}. Unique classes now: {df_proc[target_col].unique()}")

    # Check correlation matrix (only numeric columns)
    numeric = df_proc.select_dtypes(include=[np.number])
    print("\nNumeric columns considered for correlation:")
    display(numeric.columns.tolist())
    if target_col in numeric.columns:
        corr = numeric.corr()
        print("\nPearson Correlation (top correlations with target):")
        display(corr[target_col].abs().sort_values(ascending=False).head(10))
    else:
        corr = numeric.corr()
        print("\nNo numeric target found; showing correlation matrix head:")
        display(corr.head())

    # Feature selection via Pearson threshold (keep features with abs(corr with target) >= 0.1)
    if target_col in numeric.columns:
        corrs_with_target = corr[target_col].abs().drop(labels=[target_col])
        selected_features = corrs_with_target[corrs_with_target>=0.1].index.tolist()
        print("\nSelected features based on Pearson abs(corr) >= 0.1:", selected_features)
        if len(selected_features)==0:
            # fallback: take top 5 features
            selected_features = corrs_with_target.sort_values(ascending=False).head(5).index.tolist()
            print("No features passed threshold; selected top 5 instead:", selected_features)
    else:
        # If no numeric target, use all numeric features except ID cols
        selected_features = [c for c in numeric.columns if c.lower() not in ('id','patientid','pid')]
        print("Selected numeric features (no numeric target):", selected_features)

    # Analyze distribution to decide scaling
    print("\nChecking distributions (skew) for selected features:")
    skewness = df_proc[selected_features].skew().sort_values(ascending=False)
    display(skewness.head(10))
    need_scaling = any(skewness.abs()>1)
    print("Decision: Need scaling?" , need_scaling)
    scaling_note = "Applied StandardScaler" if need_scaling else "Scaling not strictly necessary; still applying StandardScaler for tree models is optional."
    print(scaling_note)

    # Prepare X and y
    if target_col:
        X = df_proc[selected_features]
        y = df_proc[target_col]
        # Drop rows with missing values in selected columns or target for simplicity
        combined = pd.concat([X,y], axis=1).dropna().reset_index(drop=True)
        X = combined[selected_features]
        y = combined[target_col]
        print("\nAfter dropping NA, data shape:", X.shape)

        # Split: first into train_val (80%) and test (20%)
        X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
        # Then split train_val into train (70% of train_val) and val (30% of train_val)
        X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.3, random_state=RANDOM_STATE, stratify=y_trainval)
        print(f"Final splits -> Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

        # Scaling if needed (fit on train, apply to val and test)
        scaler = StandardScaler()
        if need_scaling:
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            X_test_scaled = scaler.transform(X_test)
        else:
            # For trees, scaling not required, but we'll still standardize for demonstration if opted
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            X_test_scaled = scaler.transform(X_test)

        # Train a Decision Tree classifier (default depth)
        clf = DecisionTreeClassifier(random_state=RANDOM_STATE)
        clf.fit(X_train_scaled, y_train)
        y_train_pred = clf.predict(X_train_scaled)
        y_val_pred = clf.predict(X_val_scaled)
        y_test_pred = clf.predict(X_test_scaled)
        train_acc = accuracy_score(y_train, y_train_pred)
        val_acc = accuracy_score(y_val, y_val_pred)
        test_acc = accuracy_score(y_test, y_test_pred)
        print("\nDecision Tree Results:")
        print(f"Train accuracy: {train_acc:.4f}")
        print(f"Validation accuracy: {val_acc:.4f}")
        print(f"Test accuracy: {test_acc:.4f}")
        print("\nConfusion matrix on Test set:")
        display(confusion_matrix(y_test, y_test_pred))
        print("\nFeature importances (from the tree):")
        fi = pd.Series(clf.feature_importances_, index=selected_features).sort_values(ascending=False)
        display(fi)

        print("\nBrief explanations (in notebook style):")
        print("- Missing values: dropped rows with NA in selected features/target for simplicity. Alternative: imputation (mean/mode) if large dataset.\n- Categorical features were label-encoded because they were low-cardinality; for higher-cardinality or unordered categories, one-hot encoding could be used.\n- Feature selection used Pearson correlation with the target; this is a quick filter method but could miss multivariate relationships.\n- Scaling: trees don't require scaling, but StandardScaler was applied for demonstration. If using distance-based models, scaling would be necessary.\n- Validation set: used to tune hyperparameters and detect overfitting before evaluating on the held-out test set.\n")
    else:
        print("No target column available to run Task 3 model training.")

Loading dataset from: /content/cancer patient data sets.csv

First 5 rows:


,index,Patient Id,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,0,P1,33,1,2,4,5,4,3,2,...,3,4,2,2,3,1,2,3,4,Low
1,1,P10,17,1,3,1,5,3,4,2,...,1,3,7,8,6,2,1,7,2,Medium
2,2,P100,35,1,4,5,6,5,5,4,...,8,7,9,2,1,4,6,7,2,High
3,3,P1000,37,1,7,7,7,7,6,7,...,4,2,3,1,4,5,6,7,5,High
4,4,P101,46,1,6,8,7,7,7,6,...,3,2,4,1,4,2,4,2,3,High



Dataset shape: (1000, 26)

Columns and dtypes:


,0
index,int64
Patient Id,object
Age,int64
Gender,int64
Air Pollution,int64
Alcohol use,int64
Dust Allergy,int64
OccuPational Hazards,int64
Genetic Risk,int64
chronic Lung Disease,int64


Detected target column: None

Missing values per column:


...


No numeric target found; showing correlation matrix head:


,index,Patient Id,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
index,1.000000,1.000000,0.002674,-0.025739,0.053307,0.041374,0.037960,0.032355,0.030725,0.025177,...,0.042346,0.026393,0.027950,0.015078,0.005573,0.015706,0.045687,0.003793,-0.002957,-0.024556
Patient Id,1.000000,1.000000,0.002674,-0.025739,0.053307,0.041374,0.037960,0.032355,0.030725,0.025177,...,0.042346,0.026393,0.027950,0.015078,0.005573,0.015706,0.045687,0.003793,-0.002957,-0.024556
Age,0.002674,0.002674,1.000000,-0.202086,0.099494,0.151742,0.035202,0.062177,0.073151,0.128952,...,0.095059,0.106946,0.035329,-0.095354,-0.105833,0.039258,-0.012706,0.012128,-0.004700,0.042631
Gender,-0.025739,-0.025739,-0.202086,1.000000,-0.246912,-0.227636,-0.204312,-0.192343,-0.222727,-0.205061,...,-0.116467,-0.057993,-0.045972,-0.076304,-0.058324,-0.034219,-0.000526,-0.123001,-0.181618,0.086222
Air Pollution,0.053307,0.053307,0.099494,-0.246912,1.000000,0.747293,0.637503,0.608924,0.705276,0.626701,...,0.211724,0.258016,0.269558,0.055368,-0.080918,0.241065,0.174539,0.261489,-0.021343,-0.577269


Selected numeric features (no numeric target): ['index', 'Patient Id', 'Age', 'Gender', 'Air Pollution', 'Alcohol use', 'Dust Allergy', 'OccuPational Hazards', 'Genetic Risk', 'chronic Lung Disease', 'Balanced Diet', 'Obesity', 'Smoking', 'Passive Smoker', 'Chest Pain', 'Coughing of Blood', 'Fatigue', 'Weight Loss', 'Shortness of Breath', 'Wheezing', 'Swallowing Difficulty', 'Clubbing of Finger Nails', 'Frequent Cold', 'Dry Cough', 'Snoring', 'Level']

Checking distributions (skew) for selected features:


,0
Fatigue,0.855630
Clubbing of Finger Nails,0.796564
Age,0.551096
Snoring,0.550045
Swallowing Difficulty,0.451177
Passive Smoker,0.411459
Frequent Cold,0.406449
Shortness of Breath,0.406383
Gender,0.400354
Smoking,0.381312


Decision: Need scaling? False
Scaling not strictly necessary; still applying StandardScaler for tree models is optional.
No target column available to run Task 3 model training.


In [6]:
import pandas as pd
import numpy as np
from math import log2
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

RANDOM_STATE = 0

print("=== TASK 2: ID3 - Find root node from scratch ===\n")

# Toy dataset for Task 2 (Age, Job_Status, Own_House, Credit_Rating) with Target = Decision(Yes/No)
data_task2 = pd.DataFrame([
    # Age, Job_Status, Own_House, Credit_Rating, Decision
    ['Youth','Unemployed','No','Fair','No'],
    ['Youth','Unemployed','No','Excellent','No'],
    ['Middle','Employed','No','Fair','Yes'],
    ['Senior','Employed','No','Fair','Yes'],
    ['Senior','Employed','Yes','Fair','Yes'],
    ['Senior','Unemployed','Yes','Excellent','No'],
    ['Middle','Unemployed','Yes','Excellent','Yes'],
    ['Youth','Employed','No','Fair','No'],
    ['Youth','Employed','Yes','Excellent','Yes'],
    ['Senior','Employed','Yes','Excellent','Yes'],
    ['Youth','Unemployed','Yes','Fair','No'],
    ['Middle','Employed','No','Excellent','Yes'],
    ['Middle','Unemployed','No','Fair','Yes'],
    ['Senior','Unemployed','No','Excellent','No']
], columns=['Age','Job_Status','Own_House','Credit_Rating','Decision'])

display(data_task2)

def entropy(series):
    counts = series.value_counts(normalize=True)
    ent = -sum(p * log2(p) for p in counts if p>0)
    return ent

def info_gain(df, attribute, target='Decision'):
    base_entropy = entropy(df[target])
    values = df[attribute].unique()
    weighted_entropy = 0
    for v in values:
        subset = df[df[attribute]==v]
        weighted_entropy += (len(subset)/len(df)) * entropy(subset[target])
    return base_entropy - weighted_entropy

base_ent = entropy(data_task2['Decision'])
print(f"Base Entropy (Decision): {base_ent:.4f}\n")

ig_results = {}
for attr in ['Age','Job_Status','Own_House','Credit_Rating']:
    ig = info_gain(data_task2, attr, target='Decision')
    ig_results[attr] = ig
    print(f"Information Gain for {attr}: {ig:.4f}")

root_attr = max(ig_results, key=ig_results.get)
print(f"\n=> Root node chosen by ID3 (highest IG): {root_attr} (IG={ig_results[root_attr]:.4f})\n")

print("Short explanation: ID3 selects the attribute with the highest information gain as the root. Shown above are IG values.\n\n\n")


=== TASK 2: ID3 - Find root node from scratch ===



,Age,Job_Status,Own_House,Credit_Rating,Decision
0,Youth,Unemployed,No,Fair,No
1,Youth,Unemployed,No,Excellent,No
2,Middle,Employed,No,Fair,Yes
3,Senior,Employed,No,Fair,Yes
4,Senior,Employed,Yes,Fair,Yes
5,Senior,Unemployed,Yes,Excellent,No
6,Middle,Unemployed,Yes,Excellent,Yes
7,Youth,Employed,No,Fair,No
8,Youth,Employed,Yes,Excellent,Yes
9,Senior,Employed,Yes,Excellent,Yes


Base Entropy (Decision): 0.9852

Information Gain for Age: 0.3806
Information Gain for Job_Status: 0.2578
Information Gain for Own_House: 0.0202
Information Gain for Credit_Rating: 0.0000

=> Root node chosen by ID3 (highest IG): Age (IG=0.3806)

Short explanation: ID3 selects the attribute with the highest information gain as the root. Shown above are IG values.





In [5]:
data_task4 = pd.DataFrame([
    ['Alice','Yes','DataScience','Morning','Yes'],
    ['Bob','No','WebDev','Evening','No'],
    ['Carol','Yes','DataScience','Evening','Yes'],
    ['Dave','No','AI','Morning','No'],
    ['Eve','Yes','AI','Evening','Yes'],
    ['Frank','No','WebDev','Morning','No'],
    ['Grace','Yes','WebDev','Evening','Yes'],
    ['Heidi','No','DataScience','Morning','No'],
    ['Ivan','Yes','AI','Evening','Yes'],
    ['Judy','No','AI','Evening','No']
], columns=['Student','Prior_Experience','Course','Time','Enroll'])

display(data_task4.drop(columns=['Student']))

def gini(series):
    counts = series.value_counts(normalize=True)
    return 1 - sum(p*p for p in counts)

def gini_gain(df, attribute, target='Enroll'):
    base_gini = gini(df[target])
    values = df[attribute].unique()
    weighted_gini = 0
    for v in values:
        subset = df[df[attribute]==v]
        weighted_gini += (len(subset)/len(df)) * gini(subset[target])
    return base_gini - weighted_gini

base_g = gini(data_task4['Enroll'])
print(f"Base Gini (Enroll): {base_g:.4f}\n")

gini_results = {}
for attr in ['Prior_Experience','Course','Time']:
    gg = gini_gain(data_task4, attr, target='Enroll')
    gini_results[attr] = gg
    print(f"Gini Gain for {attr}: {gg:.4f}")

root_cart = max(gini_results, key=gini_results.get)
print(f"\n=> Root node chosen by CART (highest Gini gain): {root_cart} (Gini Gain={gini_results[root_cart]:.4f})\n")



,Prior_Experience,Course,Time,Enroll
0,Yes,DataScience,Morning,Yes
1,No,WebDev,Evening,No
2,Yes,DataScience,Evening,Yes
3,No,AI,Morning,No
4,Yes,AI,Evening,Yes
5,No,WebDev,Morning,No
6,Yes,WebDev,Evening,Yes
7,No,DataScience,Morning,No
8,Yes,AI,Evening,Yes
9,No,AI,Evening,No


Base Gini (Enroll): 0.5000

Gini Gain for Prior_Experience: 0.5000
Gini Gain for Course: 0.0333
Gini Gain for Time: 0.0833

=> Root node chosen by CART (highest Gini gain): Prior_Experience (Gini Gain=0.5000)

Short explanation: CART chooses splits to maximize reduction in Gini impurity; attribute with largest Gini gain is selected as root.

All tasks completed.
